In [1]:
from pymongo import MongoClient
import pandas as pd
from bson import ObjectId
from functions.location import findCityFromCoords, findCoordsFromCity

In [2]:
client = MongoClient()
db = client['companies']
companies = db['companies']
offices = db['offices']

In [4]:
#Perform a reverse geocoding query to find the city in the offices that have coordinates
startups = companies.find({'isTech':True})
available_offices = companies.find({'isAvailable':True})

for company in startups:
    for office in offices.find({'company':ObjectId(company['_id']),
                                'city':{'$in':['',None]},
                                'location.coordinates':{'$ne':[0,0]}}):
        city = findCityFromCoords(office)
        if city == 'Unknown':
            offices.delete_one(office)
        offices.update_one(office,{'$set':{'city':city}})

for company in available_offices:
    for office in offices.find({'company':ObjectId(company['_id']),
                                'city':{'$in':['',None]},
                                'location.coordinates':{'$ne':[0,0]}}):
        city = findCityFromCoords(office)
        if city == 'Unknown':
            offices.delete_one(office)
        offices.update_one(office,{'$set':{'city':city}})

In [53]:
#Perform a forward geocoding query to find the approximated coordinates from the office adress
startups = companies.find({'isTech':True})
available_offices = companies.find({'isAvailable':True})

for company in startups:
    for office in offices.find({'company':ObjectId(company['_id']),
                                'city':{'$nin':['',None]},
                                'location.coordinates':[0,0]}):
        loc = findCoordsFromCity(office)
        if loc == 'Unknown':
            offices.delete_one(office)
        offices.update_one(office,{'$set':{'location':loc}})

for company in available_offices:
    for office in offices.find({'company':ObjectId(company['_id']),
                                'city':{'$nin':['',None]},
                                'location.coordinates':[0,0]}):
        loc = findCoordsFromCity(office)
        if loc == 'Unknown':
            offices.delete_one(office)
        offices.update_one(office,{'$set':{'location':loc}})